In [82]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam, RMSprop


In [83]:
dataframe = pd.read_csv("data/raw/DAX_Data.csv", usecols=[1])
dataframe = dataframe.dropna()
#plt.plot(dataframe)
dataframe.shape

(9213, 1)

In [84]:
#Convert pandas dataframe to numpy array
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset.shape

(9213, 1)

In [85]:
scaler = MinMaxScaler(feature_range=(0, 1)) #Also try QuantileTransformer
dataset = scaler.fit_transform(dataset)
dataset.shape


(9213, 1)

In [86]:
train_size = int(len(dataset) * 0.66)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [87]:
def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset)-seq_size-1):
        #print(i)
        window = dataset[i:(i+seq_size), 0]
        x.append(window)
        y.append(dataset[i+seq_size, 0])
        
    return np.array(x),np.array(y)

In [88]:
seq_size = 5  # Number of time steps to look back 
#Larger sequences (look further back) may improve forecasting.

trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)



print("Shape of training set: {}".format(trainX.shape))
print("Shape of test set: {}".format(testX.shape))

Shape of training set: (6074, 5)
Shape of test set: (3127, 5)


In [89]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [90]:
def create_model(optimizer='adam', lstm_units=50):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(1, trainX.shape[2]), return_sequences=True))
    model.add(LSTM(lstm_units))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [91]:
# Create LSTM function for sklearn grid search 
def create_lstm_model(neurons=12, loss='mean_squared_error', activation='relu', optimizer='Adam', batch_size=12):
    model = Sequential()
    model.add(LSTM(units=neurons, activation=activation, input_shape=(n_steps_in, 1)))
    model.add(Dense(units=n_steps_out))
    model.compile(loss=loss, optimizer=optimizer, metrics=['mae'])
    
    return model

# Wrap the custom wrapper function for use with scikit-learn
lstm_regressor = KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=6, verbose=0)

# Define the parameter grid
param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adam'],
    'loss': ['mean_squared_error', 'mean_absolute_error', 'root_mean_squared_error', 'huber'],
    'batch_size': [6, 12, 18, 24],
    'epochs': [100, 250, 500],
    'neurons': [24, 36, 72],
    'activation': ['relu', 'tanh', 'sigmoid', 'linear', 'swish']}


# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=lstm_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

# Fit the grid search to the data
grid_search_result = grid_search.fit(trainX, trainY)

ValueError: Invalid parameter activation for estimator KerasRegressor.
This issue can likely be resolved by setting this parameter in the KerasRegressor constructor:
`KerasRegressor(activation=relu)`
Check the list of available parameters with `estimator.get_params().keys()`